In [248]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from joblib import dump

In [306]:
#import data and view data frame
data = pd.read_csv(
    "../Resources/player_info_reduced.csv")

data = pd.DataFrame(data)



,0304_salary_adjusted,0405_salary_adjusted,0506_salary_adjusted,0607_salary_adjusted,0708_salary_adjusted,0809_salary_adjusted,0910_salary_adjusted,1011_salary_adjusted,1112_salary_adjusted,1213_salary_adjusted,...,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,+/-
0,6482581,6748393,7041080,8511223,18546814,19514263,21677854,19711923,21033142,22654792,...,73,111,454,565,514,249,104,49,143,618
1,6482581,6748393,7041080,8511223,18546814,19514263,21677854,19711923,21033142,22654792,...,71,51,365,416,511,272,109,49,135,541
2,6136492,7499311,13711048,13362472,14577195,15369740,16279088,17128982,17206990,17566261,...,73,206,486,692,32,88,36,80,240,295
3,6136492,7499311,13711048,13362472,14577195,15369740,16279088,17128982,17206990,17566261,...,72,294,570,864,84,105,42,91,169,230
4,5208750,5422257,5657544,6855081,18546814,19514263,21677854,23313418,24309802,25114603,...,84,145,477,622,242,198,95,51,224,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0,0,0,0,0,0,0,0,0,0,...,76,22,135,157,116,50,20,3,92,-133
195,0,0,0,0,0,0,0,0,0,0,...,40,1,6,7,26,13,4,0,13,-41
196,0,0,0,0,0,0,0,0,0,0,...,73,9,52,61,55,22,26,7,42,47
197,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,-6


In [268]:
# Save Target columns 
target = data[['2526_salary_predicted', '2627_salary_predicted', '1920_salary_adjusted','2021_salary_adjusted', 
        '2122_salary_adjusted', '2223_salary_adjusted','2324_salary', '2425_salary_predicted','1819_salary_adjusted']]

# Ensure data types are appropriate
target = target.astype(float)

# isolate features
features = data.drop(columns=['Unnamed: 0', 'player_name','2526_salary_predicted', '2627_salary_predicted', 'AVG_SALARY',
                         '1920_salary_adjusted','2021_salary_adjusted', '2122_salary_adjusted', '1819_salary_adjusted', '2223_salary_adjusted',
                         '2324_salary', '2425_salary_predicted'])


# Ensure data types are appropriate
features = pd.get_dummies(features).astype(float)

target.columns

Index(['2526_salary_predicted', '2627_salary_predicted',
       '1920_salary_adjusted', '2021_salary_adjusted', '2122_salary_adjusted',
       '2223_salary_adjusted', '2324_salary', '2425_salary_predicted',
       '1819_salary_adjusted'],
      dtype='object')

In [269]:
# Create scaler instance
scaler = StandardScaler()

# # separate preprocessed data into features and target arrays
# y = data['1819_salary_adjusted']np.reshape(-1,1)
# x = data.drop(columns='1819_salary_adjusted')

# y = np.reshapeX[:,1].reshape(-1,1)

#fit standardscaler to features and then scale data
data_scaler_x = scaler.fit(features)
scaled_x = data_scaler_x.transform(features)

#fit standardscaler to targets and then scale data
data_scaler_y = scaler.fit(target)
scaled_y = data_scaler_y.transform(target)
scaled_y.shape

(199, 9)

In [271]:
#Isolate target values
y1 = scaled_y[:,8]


# split preprocessed data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y1, random_state=8)

In [274]:
# create linear regression model
reg = SGDRegressor(alpha=0.001, penalty=None)
reg.fit(X_train, y_train)


X_test

array([[-0.18699978, -0.25159379, -0.2902111 , ..., -0.52343375,
        -0.61869619,  0.06219757],
       [-0.18699978, -0.25159379, -0.2902111 , ...,  2.36742205,
         1.82998954, -0.71716385],
       [-0.18699978, -0.25159379, -0.2902111 , ..., -0.68532168,
        -1.10219465, -0.65111628],
       ...,
       [-0.18699978, -0.25159379, -0.2902111 , ...,  1.37296766,
         0.03636623, -0.41334499],
       [-0.18699978, -0.25159379, -0.2902111 , ...,  0.97981127,
         0.41068762,  0.35280691],
       [-0.18699978, -0.25159379, -0.2902111 , ...,  2.11302674,
         1.09694349,  0.71386701]])

In [275]:
# examine the error rate of our models predictions
error = y_test - reg.predict(X_test)
error

array([ 0.25740182,  0.30837484, -0.27364708, -0.13522882, -0.10539346,
       -0.52873994, -0.10847782, -0.1903548 ,  0.07858958, -0.48134944,
        0.88336772, -0.50633234, -0.22018719, -0.16492545,  0.22893598,
        1.34164566,  0.58714124, -0.00543373,  1.32906192, -0.20415893,
        0.16498251,  0.08697994, -0.18950171,  0.27209524, -0.07487214,
       -0.12340241, -0.07941524, -0.29074722,  0.28677992,  0.28507426,
        0.38859655,  0.62404495,  0.08380851, -0.42329933, -0.13065434,
        0.2094388 ,  0.60528198,  0.17125131,  0.58330295,  0.0392194 ,
       -0.3473911 , -1.47957276,  0.12836613,  0.18103815,  0.00974118,
       -0.5041073 , -0.01849908,  0.15104365,  0.32376922,  0.0249407 ])

In [276]:
# Test model
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # setting squared=False gives RMSE directly
r2 = r2_score(y_test, y_pred)

#Print Results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")


Mean Absolute Error: 0.32439931591006604
Mean Squared Error: 0.21271205897971743
Root Mean Squared Error: 0.4612071757678077
R-squared: 0.7663713714271535


['sgd_model_NBA_Salary.joblib']

## These results indicate that the model can predict with great accuracy, player salaries for the 2018-2019 season. Prediction  R^2 is high at 0.76

## Will export this as our model

## now to repeat this with minmaxscaler

In [277]:
# Adjust the data scaling 
# Create scaler instance
scaler2 = MinMaxScaler()


#fit MinMaxScaler to features and then scale data
data_scaler_2x = scaler2.fit(features)
scaled_2x = data_scaler_2x.transform(features)

#fit MinMaxScaler to targets and then scale data
data_scaler_2y = scaler2.fit(target)
scaled_2y = data_scaler_2y.transform(target)
target.columns

Index(['2526_salary_predicted', '2627_salary_predicted',
       '1920_salary_adjusted', '2021_salary_adjusted', '2122_salary_adjusted',
       '2223_salary_adjusted', '2324_salary', '2425_salary_predicted',
       '1819_salary_adjusted'],
      dtype='object')

In [278]:
#Isolate new target values
y1 = scaled_2y[:,8]

# REPEAT - separate preprocessed data into features and target arrays using re-scaled data
X_train, X_test, y_train, y_test = train_test_split(scaled_data2x, y1, random_state=22)



In [279]:
# create linear regression model
reg = SGDRegressor(alpha=0.001, penalty= None)
reg.fit(X_train, y_train)


reg

SGDRegressor(alpha=0.001, penalty=None)

In [280]:
# Test model
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred)

#Print Results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 0.07298868095667783
Mean Squared Error: 0.010261363319320487
Root Mean Squared Error: 0.010261363319320487
R-squared: 0.8465969053570933


### MinMaxScaler is slightly benficial in reducing error. begin testing on further years

In [281]:
# add 2018-2019 data to feature data
features = data.drop(columns=['Unnamed: 0', 'player_name','2526_salary_predicted', '2627_salary_predicted', 'AVG_SALARY',
                         '1920_salary_adjusted','2021_salary_adjusted', '2122_salary_adjusted', '2223_salary_adjusted',
                         '2324_salary', '2425_salary_predicted'])

In [282]:
# # Adjust the data scaling 
# # Create scaler instance
# scaler2 = MinMaxScaler()


# #fit MinMaxScaler to features and then scale data
# data_scaler_2x = scaler2.fit(features)
# scaled_2x = data_scaler_2x.transform(features)

# #fit MinMaxScaler to targets and then scale data
# data_scaler_2y = scaler2.fit(target)
# scaled_2y = data_scaler_2y.transform(target)
# target.columns

Index(['2526_salary_predicted', '2627_salary_predicted',
       '1920_salary_adjusted', '2021_salary_adjusted', '2122_salary_adjusted',
       '2223_salary_adjusted', '2324_salary', '2425_salary_predicted',
       '1819_salary_adjusted'],
      dtype='object')

In [286]:
# Create scaler instance
scaler = StandardScaler()

#fit standardscaler to features and then scale data
data_scaler_x = scaler.fit(features)
scaled_x = data_scaler_x.transform(features)

#fit standardscaler to targets and then scale data
data_scaler_y = scaler.fit(target)
scaled_y = data_scaler_y.transform(target)
scaled_y.shape

(199, 9)

In [287]:
#Isolate new target values
y2 = scaled_y[:,2]

# REPEAT - separate preprocessed data into features and target arrays using re-scaled data
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y2, random_state=71)



In [288]:
# create linear regression model 
reg = SGDRegressor(alpha=0.001, penalty= None)
reg.fit(X_train, y_train)


reg

SGDRegressor(alpha=0.001, penalty=None)

In [289]:
# Test model
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred)

#Print Results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 0.4631660009329465
Mean Squared Error: 0.3809462656276985
Root Mean Squared Error: 0.3809462656276985
R-squared: 0.6404538039237684


## Model loses accuracy (r^2 0.84 -> 0.64) but error remains low for the 2019-2020 season

In [299]:
# add 2019-2020 data to feature data
features = data.drop(columns=['Unnamed: 0', 'player_name','2526_salary_predicted', '2627_salary_predicted', 'AVG_SALARY',
                         '2021_salary_adjusted', '2122_salary_adjusted', '2223_salary_adjusted',
                         '2324_salary', '2425_salary_predicted'])

# Create scaler instance
scaler = StandardScaler()

#fit standardscaler to features and then scale data
data_scaler_x = scaler.fit(features)
scaled_x = data_scaler_x.transform(features)

#fit standardscaler to targets and then scale data
data_scaler_y = scaler.fit(target)
scaled_y = data_scaler_y.transform(target)
target.columns

Index(['2526_salary_predicted', '2627_salary_predicted',
       '1920_salary_adjusted', '2021_salary_adjusted', '2122_salary_adjusted',
       '2223_salary_adjusted', '2324_salary', '2425_salary_predicted',
       '1819_salary_adjusted'],
      dtype='object')

In [294]:
#Isolate new target values (2020-2021 season)
y2 = scaled_y[:,3]

# REPEAT - separate preprocessed data into features and target arrays using re-scaled data
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y2, random_state=9)


In [295]:
# create linear regression model 
reg = SGDRegressor(alpha=0.001, penalty= None)
reg.fit(X_train, y_train)


reg

SGDRegressor(alpha=0.001, penalty=None)

In [296]:
# Test model
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred)

#Print Results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 0.3050753932291393
Mean Squared Error: 0.19247828230235484
Root Mean Squared Error: 0.19247828230235484
R-squared: 0.8089039054982095


## R^2 jumps back up tp 0.8 for the 2020-2021 season


In [301]:
# add 2020-2021 data to feature data
features = data.drop(columns=['Unnamed: 0', 'player_name','2526_salary_predicted', '2627_salary_predicted', 'AVG_SALARY',
                          '2122_salary_adjusted', '2223_salary_adjusted',
                         '2324_salary', '2425_salary_predicted'])

# Create scaler instance
scaler = StandardScaler()

#fit standardscaler to features and then scale data
data_scaler_x = scaler.fit(features)
scaled_x = data_scaler_x.transform(features)

#fit standardscaler to targets and then scale data
data_scaler_y = scaler.fit(target)
scaled_y = data_scaler_y.transform(target)
target.columns

Index(['2526_salary_predicted', '2627_salary_predicted',
       '1920_salary_adjusted', '2021_salary_adjusted', '2122_salary_adjusted',
       '2223_salary_adjusted', '2324_salary', '2425_salary_predicted',
       '1819_salary_adjusted'],
      dtype='object')

In [302]:
#Isolate new target values (2021-2022 season)
y2 = scaled_y[:,4]

# REPEAT - separate preprocessed data into features and target arrays using re-scaled data
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y2, random_state=33)


In [303]:
# create linear regression model 
reg = SGDRegressor(alpha=0.001, penalty= None)
reg.fit(X_train, y_train)


reg

SGDRegressor(alpha=0.001, penalty=None)

In [305]:
# Test model
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred)

#Print Results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")



Mean Absolute Error: 0.2214974697916252
Mean Squared Error: 0.07522571496008623
Root Mean Squared Error: 0.07522571496008623
R-squared: 0.9291352238145903


['sgd_model_NBA_Salary.joblib']

# r^ now at 0.93 for 2021-2022 season

# lets try to predict the current season now

In [317]:
# add all previous salary data to feature data
features = data.drop(columns=['Unnamed: 0','2526_salary_predicted', '2627_salary_predicted', 'AVG_SALARY',
                         '2324_salary', '2425_salary_predicted'])



# Create scaler instance
scaler = StandardScaler()

#fit standardscaler to features and then scale data
data_scaler_x = scaler.fit(features)
scaled_x = data_scaler_x.transform(features)

#fit standardscaler to targets and then scale data
data_scaler_y = scaler.fit(target)
scaled_y = data_scaler_y.transform(target)
target.columns

In [308]:
#Isolate new target values (current salary)
y2 = scaled_y[:,6]

# REPEAT - separate preprocessed data into features and target arrays using re-scaled data
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y2, random_state=4)


In [309]:
# create linear regression model 
reg = SGDRegressor(alpha=0.001, penalty= None)
reg.fit(X_train, y_train)


reg

SGDRegressor(alpha=0.001, penalty=None)

In [310]:
# Test model
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred)

#Print Results
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

dump(reg, 'sgd_model_NBA_Salary.joblib')

Mean Absolute Error: 0.37244251647734505
Mean Squared Error: 0.44544077479445593
Root Mean Squared Error: 0.44544077479445593
R-squared: 0.5005477303117767


['sgd_model_NBA_Salary.joblib']